In [1]:
from d3m_interface import AutoML

### Generating pipelines

In [2]:
output_path = '/Users/rlopez/D3M/examples/tmp/'
train_dataset = '/Users/rlopez/D3M/examples/185_baseball/TRAIN'
test_dataset = '/Users/rlopez/D3M/examples/185_baseball/TEST'
score_dataset = '/Users/rlopez/D3M/examples/185_baseball/SCORE'

automl = AutoML(output_path, 'AlphaD3M')
pipelines = automl.search_pipelines(train_dataset, time_bound=1)

INFO: Initializing AlphaD3M AutoML...
INFO: AlphaD3M AutoML initialized!
INFO: Found pipeline id=87e725e9-09ac-4fa1-bf37-b96f3e101dbb, time=0:00:20.347129, scoring...
INFO: Scored pipeline id=87e725e9-09ac-4fa1-bf37-b96f3e101dbb, f1_macro=0.64214
INFO: Found pipeline id=c6384850-c1f3-42cf-8b3c-cad174a0595c, time=0:00:35.742241, scoring...
INFO: Scored pipeline id=c6384850-c1f3-42cf-8b3c-cad174a0595c, f1_macro=0.61677
INFO: Found pipeline id=279e8c54-4d3b-4a46-9a32-8321a66e899a, time=0:00:51.113531, scoring...
INFO: Search completed, still scoring some pending pipelines...
INFO: Scored pipeline id=279e8c54-4d3b-4a46-9a32-8321a66e899a, f1_macro=0.71535
INFO: Scoring completed for all pipelines!


In [3]:
automl.leaderboard

,ranking,id,summary,f1_macro
0,1,279e8c54-4d3b-4a46-9a32-8321a66e899a,"imputer.sklearn, encoder.dsbox, gradient_boosting.sklearn",0.71535
1,2,87e725e9-09ac-4fa1-bf37-b96f3e101dbb,"imputer.sklearn, encoder.dsbox, random_forest.sklearn",0.64214
2,3,c6384850-c1f3-42cf-8b3c-cad174a0595c,"imputer.sklearn, encoder.dsbox, extra_trees.sklearn",0.61677


In [4]:
automl.score('279e8c54-4d3b-4a46-9a32-8321a66e899a', score_dataset)

('f1_macro', 0.64322)

In [5]:
automl.export_pipeline_code('279e8c54-4d3b-4a46-9a32-8321a66e899a')

In [6]:
from d3m_interface.pipeline import Pipeline

pipeline = Pipeline()

input_data = pipeline.make_pipeline_input()

step_0 = pipeline.make_pipeline_step('d3m.primitives.data_transformation.denormalize.Common')
pipeline.connect(input_data, step_0, from_output='dataset')

step_1 = pipeline.make_pipeline_step('d3m.primitives.data_transformation.dataset_to_dataframe.Common')
pipeline.connect(step_0, step_1)

step_2 = pipeline.make_pipeline_step('d3m.primitives.data_transformation.column_parser.Common')
pipeline.connect(step_1, step_2)

step_3 = pipeline.make_pipeline_step('d3m.primitives.data_transformation.extract_columns_by_semantic_types.Common')
pipeline.set_hyperparams(step_3, exclude_columns=[], semantic_types=['https://metadata.datadrivendiscovery.org/types/Attribute'])
pipeline.connect(step_2, step_3)

step_4 = pipeline.make_pipeline_step('d3m.primitives.data_cleaning.imputer.SKlearn')
pipeline.set_hyperparams(step_4, strategy='most_frequent')
pipeline.connect(step_3, step_4)

step_5 = pipeline.make_pipeline_step('d3m.primitives.data_preprocessing.encoder.DSBOX')
pipeline.set_hyperparams(step_5, n_limit=50)
pipeline.connect(step_4, step_5)

step_6 = pipeline.make_pipeline_step('d3m.primitives.data_transformation.extract_columns_by_semantic_types.Common')
pipeline.set_hyperparams(step_6, semantic_types=['https://metadata.datadrivendiscovery.org/types/TrueTarget'])
pipeline.connect(step_1, step_6)

step_7 = pipeline.make_pipeline_step('d3m.primitives.classification.gradient_boosting.SKlearn')
pipeline.connect(step_5, step_7)
pipeline.connect(step_6, step_7, to_input='outputs')

step_8 = pipeline.make_pipeline_step('d3m.primitives.data_transformation.construct_predictions.Common')
pipeline.connect(step_1, step_8, to_input='reference')
pipeline.connect(step_7, step_8)


In [7]:
automl.score(pipeline, score_dataset)

('f1_macro', 0.64322)

In [8]:
automl.end_session()

INFO: Ending session...
INFO: Session ended!
